In [1]:
using BenchmarkTools, Compat, DataFrames, Distributions, ForwardDiff

In [2]:
# Basic trust region with truncated conjugate gradient.

# Parsed data.
df = readtable("data/parsed_model_australia.txt", separator = ' ', header = false)

const n_simul = 3

head(df)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
1,4,1,0,0,0,0,1,0,0,0,0,1,0,35,0,0,0,69,34,35,0,70,71,70,30
2,4,1,0,0,0,0,1,0,0,0,0,1,0,30,0,0,0,64,44,53,0,68,84,85,50
3,4,1,0,0,0,0,1,0,0,0,0,1,0,40,0,0,0,69,34,35,0,129,195,149,101
4,4,1,0,0,0,0,1,0,0,0,0,1,0,70,0,0,0,64,44,53,0,59,79,81,32
5,4,1,0,0,0,0,1,0,0,0,0,1,0,45,0,0,0,64,44,53,0,82,93,94,99
6,2,1,0,0,0,0,1,0,0,0,0,1,0,20,0,0,0,69,40,35,0,70,57,58,43


In [3]:
mixed_logit = DataFrame(β1 = 1.0:210.0, β2 = 1.0:210.0, β3 = 1.0:210.0)

srand(12345)

rand_contdist(Dist::Distribution) = quantile(Dist, rand())

rand_contdist (generic function with 1 method)

In [4]:
function simulate()
    for i = 1:210, j = 1:n_simul
        mixed_logit[i, j] = rand_contdist(Uniform())
    end
end

simulate()

head(mixed_logit)

,β1,β2,β3
1,0.5627138851056968,0.8499394786290626,0.37160535186424815
2,0.28336464179809084,0.381127966318632,0.36580119057192695
3,0.8350140149860443,0.26002375370524433,0.9223171788742697
4,0.040441694559945285,0.5733819926662398,0.9813641372820436
5,0.5865981007905481,0.05016684053503706,0.6117903795750386
6,0.6200985639530681,0.2157117712338983,0.5699545901561343


In [5]:
function individual(θ::Vector, i::Int64)
    m, n = size(df)
    choice = df[i, 1][1]
    data = convert(Array, df[i, 2:n])
    alternatives = collect(1:4)
    splice!(alternatives, choice)
    
    function utility(β::Vector, k::Int64)
        temp = Float64[]
        k += 1
        while k <= n
            push!(temp, df[i, k])
            k += 4
        end
        return dot(temp, β)
    end
    
    function construct(γ::Vector, θ::Vector)
        return (θ[1]+θ[2]*γ[1]) + (θ[1]+θ[2]*γ[2]) + (θ[1]+θ[2]*γ[3])
    end
    
    function logit(θ::Vector, t::Float64 = 0.0)
        γ = vec(convert(Array, mixed_logit[i, 1:3]))
        β = θ[1:4]
        k = construct(γ, θ[5:6])
        push!(β, k)
        push!(β, θ[length(θ)])
        c = utility(β, choice)
        for alternative in alternatives
            t += exp(utility(β, alternative)-c)
        end
        return 1/(1+t)
    end
    
    return logit
end

individual (generic function with 1 method)

In [6]:
function f(θ::Vector, model::Float64 = 0.0, n::Int64 = 21)
    i = 1
    while i <= n
        logit = individual(θ, i)
        model += log(logit(θ))
        i += 1
    end
    return -model/n
end

f (generic function with 3 methods)

In [7]:
immutable BasicTrustRegion{T<:Real}
    η1::T
    η2::T
    γ1::T
    γ2::T
end

function BTRDefaults()
    return BasicTrustRegion(0.01, 0.9, 0.5, 0.5)
end

type BTRState
    iter::Int64
    x::Vector
    xcand::Vector
    g::Vector
    step::Vector
    Δ::Float64
    ρ::Float64
    tol::Float64

    function BTRState()
        state = new()
        state.tol = 1e-6
        return state
    end
end

In [8]:
function acceptCandidate!(state::BTRState, b::BasicTrustRegion)
    if state.ρ >= b.η1
        return true
    else
        return false
    end
end

function updateRadius!(state::BTRState, b::BasicTrustRegion)
    if state.ρ >= b.η2
        stepnorm = norm(state.step)
        state.Δ = min(1e20, max(4*stepnorm, state.Δ))
    elseif state.ρ >= b.η1
        state.Δ *= b.γ2
    else
        state.Δ *= b.γ1
    end
end

updateRadius! (generic function with 1 method)

In [9]:
function TruncatedCG(g::Vector, H::Matrix, Δ::Float64)
    n = length(g)
    s = zeros(n)
    normg0 = norm(g)
    v = g
    d = -v
    gv = dot(g, v)
    norm2d = gv
    norm2s = 0
    sMd = 0
    k = 0
    Δ2 = Δ*Δ
    while !stopCG(norm(g), normg0, k, n)
        Hd = H*d
        κ = dot(d, Hd)
        if κ <= 0
            σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-dot(s, s))))/norm2d
            s += σ*d
            break
        end
        α = gv/κ
        norm2s += α*(2*sMd+α*norm2d)
        if norm2s >= Δ2
            σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-dot(s, s))))/norm2d
            s += σ*d
            break
        end
        s += α*d
        g += α*Hd
        v = g
        newgv = dot(g, v)
        β = newgv/gv
        gv = newgv
        d = -v+β*d
        sMd = β*(sMd+α*norm2d)
        norm2d = gv+β*β*norm2d
        k += 1
    end
    return s
end

TruncatedCG (generic function with 1 method)

In [10]:
function stopCG(normg::Float64, normg0::Float64, k::Int, kmax::Int)
    χ::Float64 = 0.1
    θ::Float64 = 0.5
    if (k == kmax) || (normg <= normg0*min(χ, normg0^θ))
        return true
    else
        return false
    end
end

stopCG (generic function with 1 method)

In [11]:
function btr(f::Function, g!::Function, H!::Function, Step::Function,
        x0::Vector, state::BTRState = BTRState(), ApproxH::Bool = false)
    b = BTRDefaults()
    state.iter = 0
    state.x = x0
    n = length(x0)
    tol2 = state.tol*state.tol
    state.g = zeros(n)
    H = eye(n, n)
    fx = f(x0)
    g!(x0, state.g)
    state.Δ = 0.1*norm(state.g)
    if ApproxH
        y = zeros(n)
        gcand = zeros(n)
    else
        H!(x0, H)
    end
    nmax = 100

    function model(s::Vector, g::Vector, H::Matrix)
        return dot(s, g)+0.5*dot(s, H*s)
    end
    
    while dot(state.g, state.g) > tol2 && state.iter < nmax
        state.step = Step(state.g, H, state.Δ)
        state.xcand = state.x+state.step
        fcand = f(state.xcand)
        state.ρ = (fcand-fx)/(model(state.step, state.g, H))
        if ApproxH
            g!(state.xcand, gcand)
            y = gcand-state.g
            H = H!(H, y, state.step)
        end
        if acceptCandidate!(state, b)
            state.x = copy(state.xcand)
            if !ApproxH
                g!(state.x, state.g)
                H!(state.x, H)
            else
                state.g = copy(gcand)
            end
            fx = fcand
        end
        updateRadius!(state, b)
        state.iter += 1
    end
    return state.x, state.iter
end

btr (generic function with 3 methods)

In [12]:
function g(x::Vector, n::Int64 = 21)
    t = zeros(length(x))
    for i = 1:n
        logit = individual(x, i)
        t += (1/logit(x))*ForwardDiff.gradient(logit, x)
    end
    return -t/n
end

function g!(x::Vector, storage::Vector)
    s = g(x)
    storage[1:length(s)] = s[1:length(s)]
end

g! (generic function with 1 method)

In [13]:
function H(x::Vector)
    return ForwardDiff.hessian(f, x)
end

function H!(x::Vector, storage::Matrix)
    s = H(x)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

H! (generic function with 1 method)

In [14]:
function BHHH(x::Vector, n::Int64 = 210)
    t = zeros(length(x), length(x))
    for i = 1:n
        logit = individual(x, i)
        g = ForwardDiff.gradient(logit, x)
        t += g*(g')
    end
    return t/n
end

function BHHH!(x::Vector, storage::Matrix)
    s = BHHH(x)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

BHHH! (generic function with 1 method)

In [15]:
function BFGS(B::Matrix, y::Vector, s::Vector)
    Bs = B*s
    return B-(Bs*Bs')/dot(s, Bs)+(y*y')/dot(s, y)
end

function BFGS!(B::Matrix, y::Vector, s::Vector)
    n, m = size(B)
    B[1:n, 1:m] = BFGS(B, y, s)
end

BFGS! (generic function with 1 method)

In [16]:
function SR1(B::Matrix, y::Vector, s::Vector)
    Bs = B*s
    return B+((y-Bs)*(y-Bs)')/((y-Bs)'*s)
end

function SR1!(B::Matrix, y::Vector, s::Vector)
    n, m = size(B)
    B[1:n, 1:m] = SR1(B, y, s)
end

SR1! (generic function with 1 method)

In [17]:
simulate()

btr(f, g!, H!, TruncatedCG, zeros(7), BTRState())

([7.44587, 10.4575, -0.176355, 0.149317, -0.0802203, -0.0297832, -0.0561553], 60)